In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import yaml
import cftime
import xarray as xr


## Define folders

In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:   path_main  = file.read()
with open('../path_CMIP5.txt', 'r') as file:  path_cmip5 = file.read()
    
dir_CMIP5   = path_cmip5
dir_scripts = f'{path_main}Scripts/'
dir_names   = f'{path_main}Scripts/Model_lists/'
dir_out1    = f'{path_main}Data/CMIP5/EMT/'
dir_out2    = f'{path_main}Data/CMIP5/GMT/'
if not os.path.exists(dir_out1):  os.mkdir(dir_out1)
if not os.path.exists(dir_out2):  os.mkdir(dir_out2)


## Prepare variables and parameters

In [ ]:
#Define limits for European box
lon_limits_EUR = [-10, 35]
lat_limits_EUR = [30, 70]

#Define scenarios
scenarios = ['historical', 'rcp85']

#Define models and RCPs which should be used
all_models = dict()
all_models['rcp85'] = []
with open(dir_names + 'Models_CMIP5_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(line[:-1])


## Calculation

In [ ]:
#Loop over models
for model in all_models['rcp85']:
    
    print(model)
    
    if model=='EC-EARTH':
        members = ['r1i1p1', 'r12i1p1']
    elif model=='MPI-ESM-LR':
        members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
    else:
        members = ['r1i1p1']

    for member in members:
    
        #Loop over scenarios
        for scen in scenarios:

            #Get file names
            dir_data = dir_CMIP5 + scen + '/'
            fnames = [dir_data + file for file in os.listdir(dir_data) if model + '_' in file and member in file and 'tas_' in file and scen in file and '_Amon_' in file]
            fnames = sorted(fnames)
            if len(fnames)==0: sys.exit('No file found for this model.')

            #Merge single files to one large file
            file_merge = dir_out1 + 'CMIP5_merged_tas_' + scen + "_" + model + '_yearmean_tmp.nc'
            if os.path.exists(file_merge): os.remove(file_merge)
            os.system('cdo mergetime ' + " ".join(fnames) + " " + file_merge)

            #Open data set and rename variable
            data = xr.open_dataset(file_merge, use_cftime=True)

            #Convert longitude from [0, 360] to [-180, 180]
            data['lon'] = data['lon'].where(data['lon']<180, ((data['lon'] + 180) % 360) - 180)
            data = data.sortby('lon')


            #Select time period
            if scen=='historical':
                data = data.sel(time=slice('1950', '2005'))
            elif scen=='rcp85':
                data = data.sel(time=slice('2006', '2100'))        

            #Calculate yearly mean
            ds_grid = data.resample(time='1Y').mean('time')

            #Get mask for Europe
            mask_EUR = ((data['lon']>=lon_limits_EUR[0]) & (data['lon']<=lon_limits_EUR[1]) & 
                        (data['lat']>=lat_limits_EUR[0]) & (data['lat']<=lat_limits_EUR[1]))

            #Mask Europe (only for EMT!)
            ds_masked = ds_grid.where(mask_EUR)

            #Calculate EMT
            area_weights = np.cos(ds_masked['lat'] * np.pi/180)
            ds_EMT = ds_masked.tas.weighted(area_weights).mean(('lat', 'lon'))
            ds_EMT = ds_EMT.to_dataset(name='tas')        

            #Calculate GMT
            area_weights = np.cos(ds_grid['lat'] * np.pi/180)
            ds_GMT = ds_grid.tas.weighted(area_weights).mean(('lat', 'lon'))  
            ds_GMT = ds_GMT.to_dataset(name='tas')        

            #Create output file names
            time_str  = str(data.time[0].dt.year.values) + '-' + str(data.time[-1].dt.year.values)
            fname_out_EMT = dir_out1 + 'EMT_year_' + scen + '_' + model + '_' + member + '_' + time_str + ".nc"
            fname_out_GMT = dir_out2 + 'GMT_year_' + scen + '_' + model + '_' + member + '_' + time_str + ".nc"

            #Save in file
            if os.path.exists(fname_out_EMT): os.remove(fname_out_EMT)
            if os.path.exists(fname_out_GMT): os.remove(fname_out_GMT)
            ds_EMT.to_netcdf(fname_out_EMT)
            ds_GMT.to_netcdf(fname_out_GMT)

            #Remove temporarily merged file
            os.remove(file_merge)


## Calculate GMT for full historical period

In [ ]:
#Loop over models
for model in all_models['rcp85']:
    
    print(model)
    
    if model=='EC-EARTH':
        members = ['r1i1p1', 'r12i1p1']
    elif model=='MPI-ESM-LR':
        members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
    else:
        members = ['r1i1p1']

    for member in members:

        #Get file names
        dir_data = dir_CMIP5 + 'historical/'
        fnames = [dir_data + file for file in os.listdir(dir_data) if model + '_' in file and member in file and 'tas_' in file and 'historical' in file and '_Amon_' in file]
        fnames = sorted(fnames)
        if len(fnames)==0: sys.exit('No file found for this model.')

        #Merge single files to one large file
        file_merge = dir_out1 + 'CMIP5_merged_tas_historical_' + model + '_yearmean_tmp.nc'
        if os.path.exists(file_merge): os.remove(file_merge)
        os.system('cdo mergetime ' + " ".join(fnames) + " " + file_merge)

        #Open data set and rename variable
        data = xr.open_dataset(file_merge, use_cftime=True)

        #Convert longitude from [0, 360] to [-180, 180]
        data['lon'] = data['lon'].where(data['lon']<180, ((data['lon'] + 180) % 360) - 180)
        data = data.sortby('lon')

        #Select time period
        data = data.sel(time=slice(None, '2005'))

        #Calculate yearly mean
        ds_grid = data.resample(time='1Y').mean('time')

        #Calculate GMT
        area_weights = np.cos(ds_grid['lat'] * np.pi/180)
        ds_GMT = ds_grid.tas.weighted(area_weights).mean(('lat', 'lon'))  
        ds_GMT = ds_GMT.to_dataset(name='tas')        

        #Create output file names
        time_str = str(data.time[0].dt.year.values) + '-' + str(data.time[-1].dt.year.values)
        fname_out_GMT = dir_out2 + 'GMT_year_historical_' + model + '_' + member + '_' + time_str + ".nc"

        #Save in file
        if os.path.exists(fname_out_GMT): os.remove(fname_out_GMT)
        ds_GMT.to_netcdf(fname_out_GMT)

        #Remove temporarily merged file
        os.remove(file_merge)
